In [55]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
import db_dtypes
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import  RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Dropout, Embedding, Concatenate

import warnings

warnings.simplefilter(action='ignore')
from pathlib import Path


In [56]:
from google.cloud import storage

# Verifica que GOOGLE_APPLICATION_CREDENTIALS esté configurado
import os
print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

# Tu código aquí
client = storage.Client()

/Users/davids./code/davidSA10/gcp/nth-fiber-448914-a7-c1adcd74a08c.json


In [57]:
client = bigquery.Client()

# Create BigQuery Storage client
bqstorage_client = bigquery_storage.BigQueryReadClient()

In [58]:

query = f"""
        with clean_data as (
                select * EXCEPT (store_number, zip_code, category, vendor_number, county_number),
                CAST(store_number AS NUMERIC) as store_number ,
                CAST(zip_code AS NUMERIC) as zip_code ,
                CAST(category AS NUMERIC) as category,
                CAST(vendor_number AS NUMERIC) as vendor_number
                from `bigquery-public-data.iowa_liquor_sales.sales`
                where date >= '2013-01-01' and date <= '2023-06-30'
                --and CAST(vendor_number AS NUMERIC) in (260,421,65,370,85,434,35,301,259,115,395,55,420,205,380,192,297,300,255,389)
                ORDER BY date ASC
        ),
        distinct_vendor as (
                select
                CAST(vendor_number AS NUMERIC) as vendor_number,
                ARRAY_TO_STRING(ARRAY_AGG(vendor_name ORDER BY date DESC LIMIT 1),"") as vendor_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ),
        distinct_category as (
                select
                CAST(category AS NUMERIC) as category,
                ARRAY_TO_STRING(ARRAY_AGG(category_name ORDER BY date DESC LIMIT 1),"") as category_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ),
        distinct_store as (
                select
                CAST(store_number AS NUMERIC) as store_number,
                ARRAY_TO_STRING(ARRAY_AGG(store_name ORDER BY date DESC LIMIT 1),"") as store_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ), clean_data2 as (
        select
                cd.* EXCEPT (vendor_name, category_name, store_name),
                dv.vendor_name,
                dc.category_name,
                ds.store_name
        from clean_data cd
        left join distinct_vendor dv on cd.vendor_number = dv.vendor_number
        left join distinct_category dc on cd.category = dc.category
        left join distinct_store ds on cd.store_number = ds.store_number
        ), group_and_others as (
        SELECT date,
        case when county in ('POLK','LINN','SCOTT','BLACK HAWK','JOHNSON') then county else 'OTHER' END AS county, #'POTTAWATTAMIE','DUBUQUE','STORY','WOODBURY','DALLAS'
        CASE
        WHEN category_name like '%RUM%' THEN 'RUM'
        WHEN category_name like '%VODKA%' THEN 'VODKA'
        WHEN category_name like '%WHISK%' or  category_name like '%SCOTCH%' THEN 'WHISKY'
        WHEN category_name like '%TEQUILA%' or category_name like '%MEZCAL%' THEN 'TEQUILA_MEZCAL'
        WHEN category_name like '%LIQUEUR%' THEN 'LIQUEURS'
        WHEN category_name like '%GIN%' THEN 'GIN'
        else 'OTROS'
        end as category_name,
        case when vendor_name in ('SAZERAC COMPANY  INC','DIAGEO AMERICAS','HEAVEN HILL BRANDS','LUXCO INC','JIM BEAM BRANDS','FIFTH GENERATION INC','PERNOD RICARD USA','MCCORMICK DISTILLING CO.','BACARDI USA INC','E & J GALLO WINERY') then vendor_name else 'OTHER' END as vendor_name,
        sum(bottles_sold) as bottles_sold
        FROM clean_data2
        group by 1,2,3,4
        ), summary as (
        select
        * EXCEPT (vendor_name)
        from group_and_others
        where lower(vendor_name) like '%bacardi%'
        ), combinations as (
        SELECT
          *
          FROM UNNEST(GENERATE_DATE_ARRAY('2013-01-01', '2023-06-30', INTERVAL 1 DAY)) as date
          cross join (select distinct category_name from summary) a
          cross join (select distinct county from summary) b
          ), data_combinations as (
        select c.*,
        date_trunc(c.date, WEEK) as date_week,
          coalesce(s.bottles_sold,0) as bottles_sold
          from combinations c
          left join summary s on c.date = s.date and c.category_name = s.category_name and c.county = s.county
          )
        select
            date_week,
            category_name, county,
            extract(YEAR FROM date_week) as week_year,
            extract(WEEK(MONDAY) from date_week) as week_of_year,
            sum(bottles_sold) as bottles_sold
        from data_combinations
        group by 1,2,3,4,5
        order by county asc, category_name asc, date_week asc

    """

# Fetch results using BigQuery Storage API
df = client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

# Display results
df.head()

,date_week,category_name,county,week_year,week_of_year,bottles_sold
0,2012-12-30,GIN,BLACK HAWK,2012,52,12
1,2013-01-06,GIN,BLACK HAWK,2013,0,11
2,2013-01-13,GIN,BLACK HAWK,2013,1,34
3,2013-01-20,GIN,BLACK HAWK,2013,2,18
4,2013-01-27,GIN,BLACK HAWK,2013,3,25


##

## Preprocess

In [59]:
month_in_year = 12
df['date_week'] = pd.to_datetime(df['date_week'])
df['num_month'] = df['date_week'].dt.month
df['sin_MoSold'] = np.sin(2*np.pi*df.num_month/month_in_year)
df['cos_MoSold'] = np.cos(2*np.pi*df.num_month/month_in_year)
df = df.drop('num_month', axis=1)
columnas_target = df[["bottles_sold"]]
columnas_apoyo = df[['category_name','county','sin_MoSold','cos_MoSold']]

In [60]:
df.head()

,date_week,category_name,county,week_year,week_of_year,bottles_sold,sin_MoSold,cos_MoSold
0,2012-12-30,GIN,BLACK HAWK,2012,52,12,-2.449294e-16,1.000000
1,2013-01-06,GIN,BLACK HAWK,2013,0,11,5.000000e-01,0.866025
2,2013-01-13,GIN,BLACK HAWK,2013,1,34,5.000000e-01,0.866025
3,2013-01-20,GIN,BLACK HAWK,2013,2,18,5.000000e-01,0.866025
4,2013-01-27,GIN,BLACK HAWK,2013,3,25,5.000000e-01,0.866025


In [61]:
# CATEGORICAL PIPE
categorical_features = ['county', 'category_name']
cat_pipe = make_pipeline(
    OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
    )
)
# NUMERIC PIPE
numerical_features = ['week_year','week_of_year','bottles_sold']
num_pipe = make_pipeline(
    RobustScaler()
)
# COMBINED PREPROCESSOR

final_preprocessor = ColumnTransformer(
    [
    ("cat_preproc", cat_pipe, categorical_features),
    ("num_preproc", num_pipe,  numerical_features)
    ],
    n_jobs=-1,
    remainder='passthrough'
)

In [62]:
X_processed = final_preprocessor.fit_transform(df)
col_names = final_preprocessor.get_feature_names_out()

In [63]:
data_processed = pd.DataFrame(
        X_processed,
        columns=col_names
)

In [64]:
print(data_processed.shape)
print(data_processed.columns)

(23016, 19)
Index(['cat_preproc__county_BLACK HAWK', 'cat_preproc__county_JOHNSON',
       'cat_preproc__county_LINN', 'cat_preproc__county_OTHER',
       'cat_preproc__county_POLK', 'cat_preproc__county_SCOTT',
       'cat_preproc__category_name_GIN', 'cat_preproc__category_name_LIQUEURS',
       'cat_preproc__category_name_OTROS', 'cat_preproc__category_name_RUM',
       'cat_preproc__category_name_TEQUILA_MEZCAL',
       'cat_preproc__category_name_VODKA', 'cat_preproc__category_name_WHISKY',
       'num_preproc__week_year', 'num_preproc__week_of_year',
       'num_preproc__bottles_sold', 'remainder__date_week',
       'remainder__sin_MoSold', 'remainder__cos_MoSold'],
      dtype='object')


In [65]:
print(data_processed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 19 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   cat_preproc__county_BLACK HAWK             23016 non-null  object        
 1   cat_preproc__county_JOHNSON                23016 non-null  object        
 2   cat_preproc__county_LINN                   23016 non-null  object        
 3   cat_preproc__county_OTHER                  23016 non-null  object        
 4   cat_preproc__county_POLK                   23016 non-null  object        
 5   cat_preproc__county_SCOTT                  23016 non-null  object        
 6   cat_preproc__category_name_GIN             23016 non-null  object        
 7   cat_preproc__category_name_LIQUEURS        23016 non-null  object        
 8   cat_preproc__category_name_OTROS           23016 non-null  object        
 9   cat_preproc__cate

In [66]:
data_processed = pd.concat([data_processed, columnas_apoyo, columnas_target], axis="columns", sort=False)
data_processed.columns = data_processed.columns.str.replace(" ", "_")

In [67]:
print(data_processed.shape)
print(data_processed.columns)

(23016, 24)
Index(['cat_preproc__county_BLACK_HAWK', 'cat_preproc__county_JOHNSON',
       'cat_preproc__county_LINN', 'cat_preproc__county_OTHER',
       'cat_preproc__county_POLK', 'cat_preproc__county_SCOTT',
       'cat_preproc__category_name_GIN', 'cat_preproc__category_name_LIQUEURS',
       'cat_preproc__category_name_OTROS', 'cat_preproc__category_name_RUM',
       'cat_preproc__category_name_TEQUILA_MEZCAL',
       'cat_preproc__category_name_VODKA', 'cat_preproc__category_name_WHISKY',
       'num_preproc__week_year', 'num_preproc__week_of_year',
       'num_preproc__bottles_sold', 'remainder__date_week',
       'remainder__sin_MoSold', 'remainder__cos_MoSold', 'category_name',
       'county', 'sin_MoSold', 'cos_MoSold', 'bottles_sold'],
      dtype='object')


In [68]:
print(data_processed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 24 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   cat_preproc__county_BLACK_HAWK             23016 non-null  object        
 1   cat_preproc__county_JOHNSON                23016 non-null  object        
 2   cat_preproc__county_LINN                   23016 non-null  object        
 3   cat_preproc__county_OTHER                  23016 non-null  object        
 4   cat_preproc__county_POLK                   23016 non-null  object        
 5   cat_preproc__county_SCOTT                  23016 non-null  object        
 6   cat_preproc__category_name_GIN             23016 non-null  object        
 7   cat_preproc__category_name_LIQUEURS        23016 non-null  object        
 8   cat_preproc__category_name_OTROS           23016 non-null  object        
 9   cat_preproc__cate

In [69]:
unique_pairs = data_processed[['county', 'category_name']].drop_duplicates().reset_index(drop=True)

In [70]:
unique_pairs

,county,category_name
0,BLACK HAWK,GIN
1,BLACK HAWK,LIQUEURS
2,BLACK HAWK,OTROS
3,BLACK HAWK,RUM
4,BLACK HAWK,TEQUILA_MEZCAL
5,BLACK HAWK,VODKA
6,BLACK HAWK,WHISKY
7,JOHNSON,GIN
8,JOHNSON,LIQUEURS
9,JOHNSON,OTROS


In [71]:
def create_sequences(df, past_steps=52, future_steps=12):
    X, y = [], []
    df_x = df.copy()  # Keep all columns, including 'num_preproc__bottles_sold'
    df_y = df[["num_preproc__bottles_sold"]].copy()  # Target variable

    for i in range(len(df) - past_steps - future_steps):
        X.append(df_x.iloc[i : i + past_steps].values)  # Past data (including target)
        y.append(df_y.iloc[i + past_steps : i + past_steps + future_steps].values)  # Future target
    return np.array(X), np.array(y)

def create_sequences_padre(data_preproc, columnas_target, past_steps=10, future_steps=1):
    assert len(data_preproc) == len(columnas_target)
    df = pd.concat([data_preproc,columnas_target], axis='columns')
    X, y = [], []
    for county in data_preproc.iloc[:,data_preproc.columns.str.contains('cat_preproc__county_')].columns:
        for cat_prod in data_preproc.iloc[:,data_preproc.columns.str.contains('cat_preproc__category_name_')].columns:
            df_filtrado = df.query(f"{county} == 1 and {cat_prod} == 1")
            X_sequence, y_sequence = create_sequences(df_filtrado,past_steps,future_steps)
            for x_item in X_sequence:
                X.append(x_item)
            for y_item in y_sequence:
                y.append([y_item])
    return np.array(X), np.array(y)


def create_sequences_padre_2(data_preproc, columnas_target, past_steps=10, future_steps=1):
    assert len(data_preproc) == len(columnas_target)
    df = pd.concat([data_preproc, columnas_target], axis='columns')  # Merge target column
    X, y = [], []
    county_columns = data_preproc.filter(like="cat_preproc__county_").columns
    category_columns = data_preproc.filter(like="cat_preproc__category_name_").columns
    for county in county_columns:
        for cat_prod in category_columns:
            df_filtrado = df[(df[county] == 1) & (df[cat_prod] == 1)]
            if df_filtrado.empty:  # Skip empty groups
                continue
            X_sequence, y_sequence = create_sequences(df_filtrado, past_steps, future_steps)
            X.extend(X_sequence)
            y.extend(y_sequence)

    return np.array(X), np.array(y)

## 

## Train

In [72]:
def initialize_model(input_shape: tuple, future_steps:int=12) -> Model:
    """
    Initialize the Neural Network with random weights
    """
    model = Sequential()

    # Better RNN layers with return_sequences=True
    #model.add(Input(shape=input_shape))
    model.add(SimpleRNN(units=64, activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(units=32, activation='tanh'))
    model.add(Dropout(0.2))
    # More complex Dense layers
    model.add(Dense(128, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(future_steps, activation="linear"))

    print("✅ Model initialized")

    return model


def compile_model(model, learning_rate=0.0005):
    """
    Compile the Neural Network
    """
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mae"])

    print("✅ Model compiled")

    return model

def train_model(model, X, y, batch_size=256,patience=2, validation_data=None, validation_split=0.3):
    """
    Fit the model and return a tuple (fitted_model, history)
    """
    print("\nTraining model...")

    es = EarlyStopping(
        monitor="val_loss",
        patience=patience,
        restore_best_weights=True,
        verbose=1
    )

    if validation_data:
        history = model.fit(
            X,
            y,
            validation_data=validation_data,
            epochs=100,
            batch_size=batch_size,
            callbacks=[es],
            verbose=1
        )
    else:
        history = model.fit(
            X,
            y,
            validation_split=validation_split,
            epochs=100,
            batch_size=batch_size,
            callbacks=[es],
            verbose=1
        )
    print(f"✅ Model trained on {len(X)} rows with min val MAE: {round(np.min(history.history['val_mae']), 2)}")
    return model, history

In [73]:
data_processed.head(10)

,cat_preproc__county_BLACK_HAWK,cat_preproc__county_JOHNSON,cat_preproc__county_LINN,cat_preproc__county_OTHER,cat_preproc__county_POLK,cat_preproc__county_SCOTT,cat_preproc__category_name_GIN,cat_preproc__category_name_LIQUEURS,cat_preproc__category_name_OTROS,cat_preproc__category_name_RUM,...,num_preproc__week_of_year,num_preproc__bottles_sold,remainder__date_week,remainder__sin_MoSold,remainder__cos_MoSold,category_name,county,sin_MoSold,cos_MoSold,bottles_sold
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.057692,-0.266862,2012-12-30,-0.0,1.0,GIN,BLACK HAWK,-2.449294e-16,1.000000e+00,12
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.942308,-0.269795,2013-01-06,0.5,0.866025,GIN,BLACK HAWK,5.000000e-01,8.660254e-01,11
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.903846,-0.202346,2013-01-13,0.5,0.866025,GIN,BLACK HAWK,5.000000e-01,8.660254e-01,34
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.865385,-0.249267,2013-01-20,0.5,0.866025,GIN,BLACK HAWK,5.000000e-01,8.660254e-01,18
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.826923,-0.228739,2013-01-27,0.5,0.866025,GIN,BLACK HAWK,5.000000e-01,8.660254e-01,25
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.788462,-0.13783,2013-02-03,0.866025,0.5,GIN,BLACK HAWK,8.660254e-01,5.000000e-01,56
6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.75,-0.14956,2013-02-10,0.866025,0.5,GIN,BLACK HAWK,8.660254e-01,5.000000e-01,52
7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.711538,-0.190616,2013-02-17,0.866025,0.5,GIN,BLACK HAWK,8.660254e-01,5.000000e-01,38
8,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.673077,-0.190616,2013-02-24,0.866025,0.5,GIN,BLACK HAWK,8.660254e-01,5.000000e-01,38
9,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.634615,-0.073314,2013-03-03,1.0,0.0,GIN,BLACK HAWK,1.000000e+00,6.123234e-17,78


In [74]:
data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 24 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   cat_preproc__county_BLACK_HAWK             23016 non-null  object        
 1   cat_preproc__county_JOHNSON                23016 non-null  object        
 2   cat_preproc__county_LINN                   23016 non-null  object        
 3   cat_preproc__county_OTHER                  23016 non-null  object        
 4   cat_preproc__county_POLK                   23016 non-null  object        
 5   cat_preproc__county_SCOTT                  23016 non-null  object        
 6   cat_preproc__category_name_GIN             23016 non-null  object        
 7   cat_preproc__category_name_LIQUEURS        23016 non-null  object        
 8   cat_preproc__category_name_OTROS           23016 non-null  object        
 9   cat_preproc__cate

In [75]:
data_processed = data_processed.drop('remainder__date_week', axis=1)
for columns in data_processed.columns:
    data_processed[columns] = data_processed[columns].astype(float)

data_processed.columns = data_processed.columns.str.replace(" ", "_")

ValueError: could not convert string to float: 'GIN'

In [52]:
batch_size = 256
split_ratio = 0.083333333
past_steps = 52
future_steps = 12
learning_rate=0.0105
patience = 10

for (county, category) in unique_pairs.itertuples(index=False):
    print(county)
    print(category)
    data_aux = data_processed[(data_processed["county"] == county) & (data_processed["category_name"] == category)]
    print(data_aux.info())
    data_aux = data_aux.iloc[:,:-(len(columnas_target.columns)+len(columnas_apoyo.columns))]
    # Sort by date to maintain time order
    data_aux = data_aux.sort_values(by="remainder__date_week").reset_index(drop=True)
    data_aux = data_processed.drop('remainder__date_week', axis=1)
    print(data_aux.info())
    print(data_aux.shape)
    print(f"creando secuencias para county={county} y category = {category}")
    X, y = create_sequences(data_aux, past_steps=past_steps, future_steps=future_steps)
    print("✅ Secuencias creadas ")
    split_index = int((1-split_ratio) * len(X))
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]
    print("✅ Train/Val Split created ")
    print("Input shape X train completo:", X_train.shape)
    print("Input shape y train completo:", y_train.shape)
    print("Input shape X val completo:", X_val.shape)
    print("Input shape y val completo:",y_val.shape)
    print(f"inicializando modelo")
    model = initialize_model(input_shape=X_train.shape[1:], future_steps=future_steps)
    model = compile_model(model, learning_rate=learning_rate)
    print("✅ Model compiled succesfully")

    model,history = train_model(
        model,
        X_train,
        y_train,
        batch_size=batch_size,
        patience=patience,
        validation_data=(X_val, y_val)
    )
    if 'val_mae' in history.history:
        val_mae = np.min(history.history['val_mae'])
    else:
        val_mae = np.min(history.history['val_loss'])  # Use validation loss instead

    print("val_mae:", val_mae)

BLACK HAWK
GIN
<class 'pandas.core.frame.DataFrame'>
Index: 548 entries, 0 to 547
Data columns (total 24 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   cat_preproc__county_BLACK_HAWK             548 non-null    object        
 1   cat_preproc__county_JOHNSON                548 non-null    object        
 2   cat_preproc__county_LINN                   548 non-null    object        
 3   cat_preproc__county_OTHER                  548 non-null    object        
 4   cat_preproc__county_POLK                   548 non-null    object        
 5   cat_preproc__county_SCOTT                  548 non-null    object        
 6   cat_preproc__category_name_GIN             548 non-null    object        
 7   cat_preproc__category_name_LIQUEURS        548 non-null    object        
 8   cat_preproc__category_name_OTROS           548 non-null    object        
 9   cat_preproc

ValueError: Invalid dtype: object

In [ ]:
split_ratio: float = 0.20 # 0.02 represents ~ 1 month of validation data on a 2009-2015 train set
learning_rate=0.0005
batch_size = 256
patience = 10

In [ ]:
len(columnas_target.columns)+len(columnas_apoyo.columns)

In [ ]:
data_processed = data_processed.iloc[:,:-(len(columnas_target.columns)+len(columnas_apoyo.columns))]
data_processed = data_processed.drop('remainder__date_week', axis=1)


In [ ]:
print(data_processed.columns)

In [ ]:
print(data_processed.shape)
print(data_processed.info())

In [ ]:
for columns in data_processed.columns:
    data_processed[columns] = data_processed[columns].astype(float)

In [ ]:
X, y = create_sequences_padre(data_processed, columnas_target, past_steps=4, future_steps=1)

In [ ]:
split_index = int((1-split_ratio) * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

In [ ]:
print("Input shape X train completo:", X_train.shape)
print("Input shape X val completo:", X_val.shape)
print("Input shape y train completo:", y_train.shape)
print("Input shape y val completo:",y_val.shape)

In [ ]:
16262+4066

In [ ]:
16262 + 4066

In [ ]:
model = initialize_model(input_shape=X_train.shape[1:])
model = compile_model(model, learning_rate=learning_rate)

In [ ]:
model,history = train_model(
    model,
    X_train,
    y_train,
    batch_size=batch_size,
    patience=patience,
    validation_data=(X_val, y_val)
)

In [ ]:
if 'val_mae' in history.history:
    val_mae = np.min(history.history['val_mae'])
else:
    val_mae = np.min(history.history['val_loss'])  # Use validation loss instead
print("val_mae:", val_mae)

## Predict

In [ ]:
query = f"""
        with clean_data as (
                select * EXCEPT (store_number, zip_code, category, vendor_number, county_number),
                CAST(store_number AS NUMERIC) as store_number ,
                CAST(zip_code AS NUMERIC) as zip_code ,
                CAST(category AS NUMERIC) as category,
                CAST(vendor_number AS NUMERIC) as vendor_number
                from `bigquery-public-data.iowa_liquor_sales.sales`
                where date >= '2013-01-01' and date <= '2025-01-31'
                --and CAST(vendor_number AS NUMERIC) in (260,421,65,370,85,434,35,301,259,115,395,55,420,205,380,192,297,300,255,389)
                ORDER BY date ASC
        ),
        distinct_vendor as (
                select
                CAST(vendor_number AS NUMERIC) as vendor_number,
                ARRAY_TO_STRING(ARRAY_AGG(vendor_name ORDER BY date DESC LIMIT 1),"") as vendor_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ),
        distinct_category as (
                select
                CAST(category AS NUMERIC) as category,
                ARRAY_TO_STRING(ARRAY_AGG(category_name ORDER BY date DESC LIMIT 1),"") as category_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ),
        distinct_store as (
                select
                CAST(store_number AS NUMERIC) as store_number,
                ARRAY_TO_STRING(ARRAY_AGG(store_name ORDER BY date DESC LIMIT 1),"") as store_name
                from `bigquery-public-data.iowa_liquor_sales.sales`
                group by 1
        ), clean_data2 as (
        select
                cd.* EXCEPT (vendor_name, category_name, store_name),
                dv.vendor_name,
                dc.category_name,
                ds.store_name
        from clean_data cd
        left join distinct_vendor dv on cd.vendor_number = dv.vendor_number
        left join distinct_category dc on cd.category = dc.category
        left join distinct_store ds on cd.store_number = ds.store_number
        ), group_and_others as (
        SELECT date,
        case when county in ('POLK','LINN','SCOTT','BLACK HAWK','JOHNSON') then county else 'OTHER' END AS county, #'POTTAWATTAMIE','DUBUQUE','STORY','WOODBURY','DALLAS'
        CASE
        WHEN category_name like '%RUM%' THEN 'RUM'
        WHEN category_name like '%VODKA%' THEN 'VODKA'
        WHEN category_name like '%WHISK%' or  category_name like '%SCOTCH%' THEN 'WHISKY'
        WHEN category_name like '%TEQUILA%' or category_name like '%MEZCAL%' THEN 'TEQUILA_MEZCAL'
        WHEN category_name like '%LIQUEUR%' THEN 'LIQUEURS'
        WHEN category_name like '%GIN%' THEN 'GIN'
        else 'OTROS'
        end as category_name,
        case when vendor_name in ('SAZERAC COMPANY  INC','DIAGEO AMERICAS','HEAVEN HILL BRANDS','LUXCO INC','JIM BEAM BRANDS','FIFTH GENERATION INC','PERNOD RICARD USA','MCCORMICK DISTILLING CO.','BACARDI USA INC','E & J GALLO WINERY') then vendor_name else 'OTHER' END as vendor_name,
        sum(bottles_sold) as bottles_sold
        FROM clean_data2
        group by 1,2,3,4
        ), summary as (
        select
        * EXCEPT (vendor_name)
        from group_and_others
        where lower(vendor_name) like '%bacardi%'
        ), combinations as (
        SELECT
          *
          FROM UNNEST(GENERATE_DATE_ARRAY('2013-01-01', '2025-01-31', INTERVAL 1 DAY)) as date
          cross join (select distinct category_name from summary) a
          cross join (select distinct county from summary) b
          ), data_combinations as (
        select c.*,
        date_trunc(c.date, WEEK) as date_week,
          coalesce(s.bottles_sold,0) as bottles_sold
          from combinations c
          left join summary s on c.date = s.date and c.category_name = s.category_name and c.county = s.county
          )
          select date_week, category_name, county,
          extract(YEAR FROM date_week) as week_year,
          extract(WEEK(MONDAY) from date_week) as week_of_year,
           sum(bottles_sold) as bottles_sold
           from data_combinations
           group by 1,2,3,4,5
           order by county asc, category_name asc, date_week asc

    """

# Fetch results using BigQuery Storage API
df_pred = client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

# Display results
df_pred.head()

In [ ]:
month_in_year = 12
df_pred['date_week'] = pd.to_datetime(df_pred['date_week'])
df_pred['num_month'] = df_pred['date_week'].dt.month
df_pred['sin_MoSold'] = np.sin(2*np.pi*df_pred.num_month/month_in_year)
df_pred['cos_MoSold'] = np.cos(2*np.pi*df_pred.num_month/month_in_year)
df_pred = df_pred.drop('num_month', axis=1)
X_pred = final_preprocessor.transform(df_pred)
col_names_pred = final_preprocessor.get_feature_names_out()

In [ ]:
X_pred_processed = pd.DataFrame(
    X_pred,
    columns=col_names_pred
)

In [ ]:
X_pred_processed = X_pred_processed.drop('remainder__date_week', axis=1)
for columns in X_pred_processed.columns:
    X_pred_processed[columns] = X_pred_processed[columns].astype(float)

X_pred_processed.columns = X_pred_processed.columns.str.replace(" ", "_")
print(X_pred_processed.shape)
X_pred_processed.info()

In [ ]:
def create_sequences_inference(data_preproc, past_steps=52):
    """
    Create sequences from new unseen data for inference (prediction).
    Returns only X_pred (input features), without y.
    """
    X_pred = []
    # Ensure that we have at least 'past_steps' weeks of data
    if len(data_preproc) < past_steps:
        raise ValueError(f"Not enough data. Need at least {past_steps} weeks, got {len(data_preproc)}")

    for county in data_preproc.iloc[:, data_preproc.columns.str.contains('cat_preproc__county_')].columns:
        for cat_prod in data_preproc.iloc[:, data_preproc.columns.str.contains('cat_preproc__category_name_')].columns:
            df_filtrado = data_preproc.query(f"{county} == 1 and {cat_prod} == 1")

            # Extract the last 'past_steps' weeks
            if len(df_filtrado) >= past_steps:
                X_pred.append(df_filtrado.iloc[-past_steps:].values)  # Last x weeks

    return np.array(X_pred)  # Shape: (num_groups, past_steps, num_features)


In [ ]:
X_pred_final = create_sequences_inference(X_pred_processed, past_steps=4)

In [ ]:
y_pred = model.predict(X_pred_final)

In [ ]:
y_pred.shape

In [ ]:
print(y_pred)

In [ ]:
def predict_future(model, last_sequence, future_steps=3):
    """
    Predict future sales for 'future_steps' weeks using autoregressive forecasting.
    """
    future_preds = []
    current_seq = last_sequence.copy()  # Start with the last available sequence

    for _ in range(future_steps):
        # Make prediction (expecting shape (1, 52, 18))
        next_pred = model.predict(current_seq.reshape(1, *current_seq.shape), verbose=0)

        # Extract prediction (assuming it's a single value per step)
        next_pred_value = next_pred.flatten()[0]
        future_preds.append(next_pred_value)

        # Create new row with predicted value and keep other features unchanged
        new_row = current_seq[-1].copy()  # Take last row as reference
        new_row[0] = next_pred_value  # Replace the bottles_sold value (assuming it's the first column)

        # Shift sequence and insert new prediction
        current_seq = np.roll(current_seq, shift=-1, axis=0)
        current_seq[-1] = new_row  # Replace last row with updated prediction

    return future_preds


In [ ]:
y_future = predict_future(model, X_pred_final, future_steps=2)

In [ ]:
import matplotlib.pyplot as plt

# Example: Plot for a specific country and category
country_to_plot = "USA"
category_to_plot = "Whiskey"

df_plot = df_combined[(df_combined['country'] == country_to_plot) &
                      (df_combined['category'] == category_to_plot)]

plt.figure(figsize=(12, 6))
plt.plot(df_plot['date_week'], df_plot['bottles_sold'], label="Actual Sales", marker='o')
plt.plot(df_plot['date_week'], df_plot['bottles_sold_predicted'], label="Predicted Sales", linestyle="dashed", marker='x')

plt.xlabel("Date")
plt.ylabel("Bottles Sold")
plt.title(f"Sales Forecast for {country_to_plot} - {category_to_plot}")
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.show()